In [13]:
!pip install argparse

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [14]:
!pip install argparse

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 36.1 MB/s eta 0:00:00


In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from tqdm import tqdm
import torch.optim as optim
import math
import time
from datetime import datetime
import shutil
import tensorboard
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF
from PIL import Image, ImageDraw, ImageFont
import torchvision.utils as vutils
from torchvision.transforms.functional import to_pil_image
from sklearn.metrics import confusion_matrix
import itertools

In [5]:
import argparse

# Add any other command-line arguments that your script needs
parser = argparse.ArgumentParser(description="prediction script")
parser.add_argument('--chkpoint', type=str, help='Path to model saved checkpoint', required=True)
parser.add_argument('--image_folder', type=str, help='Path to input image folder', required=True)
parser.add_argument('--save_dir', type=str, help='Path to output Image folders', required=True)
parser.add_argument('--tensorboard_dir', type=str, help='Path to save tensorboard dir', required=True)

_StoreAction(option_strings=['--tensorboard_dir'], dest='tensorboard_dir', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=True, help='Path to save tensorboard dir', metavar=None)

In [6]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] --chkpoint CHKPOINT --image_folder
                             IMAGE_FOLDER --save_dir SAVE_DIR
                             --tensorboard_dir TENSORBOARD_DIR
ipykernel_launcher.py: error: the following arguments are required: --chkpoint, --image_folder, --save_dir, --tensorboard_dir


SystemExit: 2

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
batch_size=16
chkpoint=args.chkpoint
#"/home/ec2-user/SageMaker/checkpoints/uncropped/Model_v2_binary_best.pth.tar"
dropout=0.2
image_folder=args.image_folder
#'/home/ec2-user/SageMaker/prediction_July4/Dataset/'
tensorboard_dir = args.tensorboard_dir
# '/home/ec2-user/SageMaker/prediction_July4/runs/'
save_dir=save_dir
#'/home/ec2-user/SageMaker/prediction_July4/'

In [10]:
class ZooplanktonDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_names = [name for name in os.listdir(image_folder) if name != ".ipynb_checkpoints"]
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, index):
        img_name = self.image_names[index]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_name

In [11]:
data_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ZooplanktonDataset(image_folder=image_folder,
                            transform=data_transform)

test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

NameError: name 'image_folder' is not defined

In [12]:
class Model_v2(nn.Module):
    def __init__(self, dropout):
        super(Model_v2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(3, 2)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout)
        
        # Adjusting the fully connected layers to fit the spatial dimensions
        # of the input image (256x256)
        self.fc1 = nn.Linear(64 * 30 * 30, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 1)  # Output layer for binary classification (1 neuron for probability)
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for probability output
        
    def forward(self, x):
        x = self.dropout(F.relu(self.conv1(x)))
        x = self.pool1(x)
        x = self.dropout(F.relu(self.conv2(x)))
        x = self.pool2(x)
        x = self.dropout(F.relu(self.conv3(x)))
        x = self.pool3(x)
        x = x.view(-1, 64 * 30 * 30)  # Adjusted view shape
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.sigmoid(self.fc3(x))  # Use sigmoid activation for probability output
        return x

In [ ]:
class Trainer:
    def __init__(self, model, test_data_loader, save_dir, with_cuda,tensorboard_dir, print_freq=1):
        self.device = torch.device("cuda:0" if with_cuda else "cpu")
        self.model = model
        self.test_data_loader = test_data_loader
        self.save_dir = save_dir
        self.print_freq = print_freq
        self.step = 0
        self.accuracy = 0
        self.tensorboard_dir = tensorboard_dir
        self.start_time = datetime.now().strftime('%b-%d_%H-%M')
        if not os.path.exists(self.tensorboard_dir):
            os.makedirs(self.tensorboard_dir)
        self.writer = SummaryWriter(log_dir=self.tensorboard_dir)

    def predict(self, threshold=0.5):
        checkpoint = torch.load(chkpoint)
        self.model.load_state_dict(checkpoint['state_dict'])
        self.model.eval()
        images = []
        predicted_labels = []
        names = []

        for batch_images, image_names in self.test_data_loader:
            with torch.no_grad():
                if self.device:
                    batch_images = batch_images.to(self.device)

                # Forward pass to obtain predicted probabilities for each item in the batch
                batch_outputs = self.model(batch_images)
                batch_predicted_labels = (batch_outputs >= threshold).int()  # Apply threshold for binary classification

            # Iterate over each item in the batch
            for i in range(batch_images.size(0)):
                image = vutils.make_grid(batch_images[i], nrow=1, padding=10, normalize=True)
                predicted_label = batch_predicted_labels[i].item()  # Convert to scalar value
                name = image_names[i]

                images.append(image)
                predicted_labels.append(predicted_label)
                names.append(name)
        
        
        zooplankton_dir = os.path.join(self.save_dir, 'Detected')
        marine_snow_dir = os.path.join(self.save_dir, 'Not-Detected')

        # Delete directories if they exist
        if os.path.exists(zooplankton_dir):
            shutil.rmtree(zooplankton_dir)
        if os.path.exists(marine_snow_dir):
            shutil.rmtree(marine_snow_dir)

        # Create directories
        os.makedirs(zooplankton_dir)
        os.makedirs(marine_snow_dir)

        # Save the classified images into respective directories based on their predicted labels
        for i in range(len(images)):
            image = images[i]
            predicted_label = predicted_labels[i]
            image_name = names[i]
            # Save the image to the appropriate directory based on the predicted label
            if predicted_label == 1:
                save_path = os.path.join(zooplankton_dir, image_name)
            else:
                save_path = os.path.join(marine_snow_dir, image_name)

            vutils.save_image(image, save_path, normalize=True)

        # Log the images on TensorBoard
        for i, image in enumerate(images):
            image_name = names[i]
            label = predicted_labels[i]
            self.writer.add_image(f"Image-{i}-Predicted-{label}", image.cpu(), global_step=i)

        # Close the SummaryWriter object
        self.writer.close()

In [ ]:
    #model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Model_v2(dropout)
    model.to(device)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    #creating object for tensorboard_dir
    tensorboard_dir = tensorboard_dir

In [ ]:
 trainer = Trainer(
        model,
        test_data_loader=test_loader,
        save_dir=save_dir,
        with_cuda=True,
        tensorboard_dir=tensorboard_dir)

In [ ]:
trainer.predict(threshold=0.1)

# Make csv

In [ ]:
import os
import csv
import shutil

folder1_path = '/home/ec2-user/SageMaker/prediction_July4/Detected'
folder2_path = '/home/ec2-user/SageMaker/prediction_July4/Not-Detected'
csv_file_path = '/home/ec2-user/SageMaker/prediction_July4/UnModified_prediction.csv'

folder1_images = os.listdir(folder1_path)
folder2_images = os.listdir(folder2_path)

data = []
for image_name in folder1_images:
    data.append([image_name, 'Detected'])

for image_name in folder2_images:
    data.append([image_name, 'NotDetected'])

with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Image Name', 'Label'])
    writer.writerows(data)

print(f"CSV file created: {csv_file_path}")

# Comparing and creating confusion matrix

In [ ]:
df1 = pd.read_csv('/home/ec2-user/SageMaker/prediction_July4/2023-07-06_Natasha.csv')
df2 = pd.read_csv('/home/ec2-user/SageMaker/prediction_July4/UnModified_prediction.csv')
df1 = df1.iloc[:, [1, 9]]
df1 = df1.rename(columns={'2: Video or Image Identifier': 'Image Name','10-11+: Repeated Species':'Label'})
df1 = df1.drop_duplicates(subset='Image Name').reset_index(drop=True)
df1['Label'] = 'Detected'
merged_df = df1.merge(df2, on='Image Name', how='outer')
merged_df['Label_x'].fillna('Not-Detected', inplace=True)
confusion_matrix = merged_df.groupby(['Label_x', 'Label_y']).size().unstack(fill_value=0)

# Calculate accuracy
correct_predictions = np.diag(confusion_matrix).sum()
total_predictions = confusion_matrix.values.sum()
accuracy = correct_predictions / total_predictions * 100
print(f'Accuracy: {accuracy:.2f}')

#visulaization

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
df1 = pd.read_csv('/home/ec2-user/SageMaker/prediction_July4/2023-07-06_Natasha.csv')
df2 = pd.read_csv('/home/ec2-user/SageMaker/prediction_July4/UnModified_prediction.csv')
df1 = df1.iloc[:, [1, 9]]
print(df1.shape)
df1 = df1.rename(columns={'2: Video or Image Identifier': 'Image Name','10-11+: Repeated Species':'Label'})
values_to_drop = ['hydrozoa-maybe', 'copepod-maybe', 'copepod-maybe-edge', 'echinoderm-larva-maybe', 'unknown']
df1 = df1[~df1.iloc[:, 1].isin(values_to_drop)]
df1 = df1.drop_duplicates(subset='Image Name').reset_index(drop=True)
print(df1.shape)
df1['Label'] = 'Detected'
merged_df = df1.merge(df2, on='Image Name', how='outer')
merged_df['Label_x'].fillna('Not-Detected', inplace=True)
confusion_matrix = merged_df.groupby(['Label_x', 'Label_y']).size().unstack(fill_value=0)

# Calculate accuracy
correct_predictions = np.diag(confusion_matrix).sum()
total_predictions = confusion_matrix.values.sum()
accuracy = correct_predictions / total_predictions * 100
print(f'Accuracy: {accuracy:.2f}')

#visulaization

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve,auc
# Convert labels to binary values
merged_df['Label_y'] = np.where(merged_df['Label_y'] == 'Detected', 1, 0)
merged_df['Label_x'] = np.where(merged_df['Label_x'] == 'Detected', 1, 0)

# Calculate TPR and FPR
tn, fp, fn, tp = confusion_matrix.values.ravel()
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(merged_df['Label_y'], merged_df['Label_x'])
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()
